In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from Helper import (Chapters_Intros, generate_llama_response, ProcessText, chunk_chapters, retrieve_chunks,  load_chroma_databases, retrieve_relevant_chunks,Save_ChromaDb, retrieve_relevant_chunks_Pass_Chapters_To_BestChunks )

ModuleNotFoundError: No module named 'langchain_huggingface'

#  CONFIGURATION  #


In [ ]:
pdf_path = "2018_IRC_1stptg.pdf"
chunk_size = 800
chunk_overlap = 150
model_name = "sentence-transformers/all-MiniLM-L6-v2"
top_k_per_chapter=5
top_k=10
#query = "How do you determine the maximum gas demand and properly size gas piping"
query = "what materials are acceptable for fireblocking in residential construction?"


In [3]:

Chapters = {
    1:[26,34], 2:[36,52], 3:[54,112], 4:[114,159], 5:[160,197], 6:[198,383], 7:[384,407], 8:[408,454], 9:[456,470], 
    10:[472,482], 11:[484,519], 12:[520,521], 13:[522,527], 14:[528,532], 15:[534,537], 16:[538,541], 17:[542,543], 
    18:[544,547], 19:[548,549], 20:[550,551], 21:[552,557], 22:[558,559], 23:[560,561], 24:[562,640], 25:[642,643], 
    26:[644,647], 27:[648,653], 28:[654,655], 29:[656,687], 30:[688,700], 31:[702,707], 32:[708,709], 33:[710,711], 
    34:[712,719], 35:[720,724], 36:[726,736], 37:[738,745], 38:[746,750], 39:[752,779], 40:[780,785], 41:[786,788], 
    42:[790,802], 43:[804,806], 44:[808,987],
}
chapters_to_chunk = list(Chapters.keys())


#  PROCESSING PDF  #

In [4]:
print(" Processing PDF to extract chapter texts and tables...")
#ProcessText(pdf_path, Chapters)
print(" PDF processing complete.")

 Processing PDF to extract chapter texts and tables...
 PDF processing complete.


#  CHUNKING CHAPTER TEXT  #

In [5]:
print(" Chunking chapter texts for efficient retrieval...")
#chunk_chapters(chapters_to_chunk,type="text", chunk_size=chunk_size, chunk_overlap=chunk_overlap)
print(" Chunking complete.")

 Chunking chapter texts for efficient retrieval...
 Chunking complete.


In [6]:
print(" Chunking chapter Table for efficient retrieval...")
#chunk_chapters(chapters_to_chunk,type="table", chunk_size=chunk_size, chunk_overlap=chunk_overlap)
print("Chunking complete.")

 Chunking chapter Table for efficient retrieval...
Chunking complete.


##  RETRIEVING CHUNKS FROM DISK  #

In [7]:
print(" Retrieving Text chunked data from disk...")
Retrieved_Text_Chunks = retrieve_chunks(chunk_size,chunk_overlap,"text")
print(f" Retrieved {len(Retrieved_Text_Chunks)} chapters worth of chunked data.")

 Retrieving Text chunked data from disk...
Retrieved chunks for 44 chapters from Pkl_Files/Text_Chunks\800_150.
 Retrieved 44 chapters worth of chunked data.


In [8]:
print(" Retrieving Table chunked data from disk...")
Retrieved_Table_Chunks = retrieve_chunks(chunk_size, chunk_overlap,"table")
print(f" Retrieved {len(Retrieved_Table_Chunks)} chapters worth of chunked data.")

 Retrieving Table chunked data from disk...
Retrieved chunks for 44 chapters from Pkl_Files/Table_Chunks\800_150.
 Retrieved 44 chapters worth of chunked data.


##  CREATING VECTOR DATABASES  #


In [9]:
print(" Creating vector databases for each chapter...")
Model = HuggingFaceEmbeddings(model_name=model_name)  

 Creating vector databases for each chapter...


c:\Users\jayas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#Text_Chunks_ChromaDb_Langchain = Save_ChromaDb(Retrieved_Text_Chunks,"text",Model,model_name,chunk_size,chunk_overlap)

In [11]:
#Table_Chunks_ChromaDb_Langchain = Save_ChromaDb(Retrieved_Table_Chunks,"table",Model,model_name,chunk_size,chunk_overlap)

#  LOADING SELECTED CHAPTERS FROM DATABASE  #

In [ ]:
#query = "How do you determine the maximum gas demand and properly size gas piping"
#query = "What are the guardrail requirements for elevated surfaces?"
query = "what materials are acceptable for fireblocking in residential construction?"

selected_chapters_UnProcessed=retrieve_relevant_chunks_Pass_Chapters_To_BestChunks(f'./ChromaDB_Query/Intro_SelectChapters',Chapters_Intros,Model,query,5)

In [ ]:
selected_chapters = [doc.metadata['chapter'] for doc in selected_chapters_UnProcessed]
print(f" Loading selected chapters: {selected_chapters}")

SelectedChapters_Text = load_chroma_databases(model_name,"text",chunk_size,chunk_overlap,selected_chapters=selected_chapters)
print(" Selected Text chapters loaded.")

In [ ]:
SelectedChapters_Table = load_chroma_databases(model_name,"table",chunk_size,chunk_overlap,selected_chapters=selected_chapters)
print(" Selected Table chapters loaded.")

#  RETRIEVING RELEVANT CHUNKS  #


In [26]:
print(f" Retrieving relevant Text chunks for query: '{query}'")
Docs_Score_Text = retrieve_relevant_chunks(query, SelectedChapters_Text, top_k_per_chapter)
if(len(Docs_Score_Text)>5):
    print("Passing to second filter")
    Docs_Score_Text=retrieve_relevant_chunks_Pass_Chapters_To_BestChunks('ChromaDB_Query/Text',Docs_Score_Text,Model,query,top_k)

 Retrieving relevant Text chunks for query: 'How do you determine the maximum gas demand and properly size gas piping'
Passing to second filter


In [27]:
print(f" Retrieving relevant Table chunks for query: '{query}'")
Docs_Score_Table = retrieve_relevant_chunks(query, SelectedChapters_Table, top_k_per_chapter)
if(len(Docs_Score_Table)>5):
    print("Passing to second filter")
    Docs_Score_Table = retrieve_relevant_chunks_Pass_Chapters_To_BestChunks('./ChromaDB_Query/Table',Docs_Score_Table,Model,query,top_k)

 Retrieving relevant Table chunks for query: 'How do you determine the maximum gas demand and properly size gas piping'
Passing to second filter


In [28]:
Retrieved_Text = " ".join([doc.page_content for doc in Docs_Score_Text])
print(" Retrieved relevant text.")

Retrieved_Table = " ".join([doc.page_content for doc in Docs_Score_Table])
print(" Retrieved relevant Table.")

 Retrieved relevant text.
 Retrieved relevant Table.


#  DISPLAYING OUTPUT  #

In [29]:
print("\n FINAL RETRIEVED TEXT:")
#print(Retrieved_Text)


 FINAL RETRIEVED TEXT:


In [30]:
print("\n FINAL RETRIEVED Table:")
#print(Retrieved_Table)


 FINAL RETRIEVED Table:


In [33]:
#print(generate_llama_response(Retrieved_Text,Retrieved_Table, query))
print(generate_llama_response(Retrieved_Text,"NO TABLES", query))

According  to  the  provided  content ,  Section  G 241 2  states  that :

 *  The  total  connected  hourly  load  shall  be  used  as  the  basis  for  pipe  sizing ,  assuming  that  all  appliances  could  be  operating  at  full  capacity  simultaneously .
 *  The  maximum  gas  demand  is  determined  by  using  this  total  connected  hourly  load .

 To  properly  size  gas  piping ,  you  should  follow  these  steps :

 1 .  Determine  the  total  connected  hourly  load  of  all  appliances  and  devices  connected  to  the  gas  piping  system .
 2 .  Use  this  load  as  the  basis  for  pipe  sizing ,  assuming  that  all  appliances  could  be  operating  at  full  capacity  simultaneously .

 This  is  stated  in  Section  G 241 3 . 4 ( 402 . 4 )  S izing  tables  and  equations :

 "This  section  applies  to  piping  materials  other  than  non corr ug ated  stainless  steel  tubing .  Where  Tables  G 241 3 . 4 ( 1 )  through  G 241 3 . 4 ( 21 )  are  used  to  size 